In [1]:
# Install dependencies
%pip install -U --force-reinstall llama-index llama-index-embeddings-huggingface llama-index-llms-llama-cpp guidance transformers[torch] ragas datasets pandas langchain-core langchain-community sentence-transformers renumics-spotlight
# %pip install llama-cpp-python # without cuda support
!PATH="/usr/local/cuda-12.3/bin:$PATH" CMAKE_ARGS="-DLLAMA_CUBLAS=on" pip install --force-reinstall --no-cache llama-cpp-python

  Obtaining dependency information for llama-index from https://files.pythonhosted.org/packages/25/1d/93884eb65bceaa46bbbc296c0564dfe69372ad88c155cf6638b3762b91de/llama_index-0.10.20-py3-none-any.whl.metadata
  Obtaining dependency information for llama-index-embeddings-huggingface from https://files.pythonhosted.org/packages/d5/ee/db2cf547e2f40e801a839ef9bbd5a2f6cc74894b5ae8bdfc21cb8dde273c/llama_index_embeddings_huggingface-0.1.4-py3-none-any.whl.metadata
  Using cached llama_index_embeddings_huggingface-0.1.4-py3-none-any.whl.metadata (806 bytes)
  Obtaining dependency information for llama-index-llms-llama-cpp from https://files.pythonhosted.org/packages/6e/f5/149f43c6eb831c864ccae37611a5bf996eb89da39fa04f5ff7496bf87a42/llama_index_llms_llama_cpp-0.1.3-py3-none-any.whl.metadata
  Using cached llama_index_llms_llama_cpp-0.1.3-py3-none-any.whl.metadata (695 bytes)
  Obtaining dependency information for guidance from https://files.pythonhosted.org/packages/24/f0/4dc780f7b57209cb915b7d

UsageError: Line magic function `%PATH="/usr/local/cuda-12.3/bin:$PATH"` not found.


In [ ]:
# Download the model
!huggingface-cli download TheBloke/Mistral-7B-Instruct-v0.2-GGUF mistral-7b-instruct-v0.2.Q4_K_M.gguf --local-dir . --local-dir-use-symlinks False

In [1]:
# Our basic llama-index RAG setup
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    # set_global_handler
)
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.llama_cpp import LlamaCPP
from llama_index.llms.llama_cpp.llama_utils import (
    messages_to_prompt,
    completion_to_prompt,
)

# Debugging for checking Context and LLM Output
# set_global_handler("simple")

# Initialize the LLM (e.g., LlamaCPP with your model)
llm = LlamaCPP(
    model_path="./mistral-7b-instruct-v0.2.Q4_K_M.gguf",
    temperature=0.0,
    max_new_tokens=2048,
    context_window=4096,
    generate_kwargs={},
    model_kwargs={"n_gpu_layers": -1},
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    verbose=False
)

# Initialize the embedding model
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")



# Load documents
documents = SimpleDirectoryReader("./data", recursive=True).load_data()

# Create an index from the documents using the service context
index = VectorStoreIndex.from_documents(documents, embed_model=embed_model)

# Now you can use the index to perform queries
query_engine = index.as_query_engine(llm=llm, similarity_top_k=3,  chunk_size=256, chunk_overlap=64)

/home/daniel/code/llamaindex-rag-glossary/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
ggml_init_cublas: GGML_CUDA_FORCE_MMQ:   no
ggml_init_cublas: CUDA_USE_TENSOR_CORES: yes
ggml_init_cublas: found 1 CUDA devices:
  Device 0: NVIDIA GeForce RTX 3060, compute capability 8.6, VMM: yes


In [2]:
# Testing the RAG Setup with an example question
response = query_engine.query("What is the similarity map?")
print(response)

 The similarity map is a data visualization component that projects datapoints onto a 2D map based on the similarity of selected columns. The similarity can be measured by pre-computed descriptions like embeddings or by one or multiple scalar columns in the dataset. Users can select columns to 'placeBy' in the settings to arrange the data on the map using UMAP or PCA algorithms. The map can be moved, zoomed, and selected points can be added or removed using mouse controls. Coloring and sizing can also be used to represent more columns on the map. The similarity map can account for similarities of filtered out points by default but can be configured to compute similarities only on filtered points. The similarity map can be added to the Spotlight layout using the `similaritymap()` function in the API.


In [3]:
# Evaluating our current pipeline and identifying shortcomings.
EXAMPLE_QUESTIONS_WITH_GT = [
    {"question": "What is the similarity map?",
     "answer": "The similarity map is a dimensionality reduction plot that displays high dimensional data like embeddings in 2D space. It can be very useful for getting a first overview on dataset structure and possible problems, before diving into more detailed analysis."
    },
    {
        "question": "How can I install Renumics Spotlight?",
        "answer": "Just run pip install renumics-spotlight. After that, start it by running spotlight from the CLI or use the Python interface via spotlight.show."
    },
    {
        "question": "I want to find duplicates in my data. How can I do this?",
        "answer": "You can either just do that exploratively, e.g. by placing data on the similarity map and checking if nearby datapoints are identical via the inspector. You can also detect duplicates with specialized tooling like annoy and display the results in spotlight via spotlight.show."
    },
    {
        "question": "Which possibilities regarding cluster analysis does Spotlight offer and how can it help me?",
        "answer": "Spotlight offers the similarity map, which can display embeddings or feature arrays in 2D. You can then identify clusters visually and browse through them while looking at the underlying data such as images or audio files in the inspector."
    },
    {
        "question": "I want to identify label issues for my audio use case? Which features and visualizations can help me and how could the process look?",
        "answer": "Similarity Map, Inspector and Filtering can help you with that. You could simply place your data points on the similarity map using an general purpose audio embedding. You could then color the data points by label. If there are clusters with mixed labels, that's a sign for label issues. You could also train a model on the existing labels and then color by model error, which is a great way of pinpointing the issues even faster."
    },
    {
        "question": "What are the main UI elements that help me with my use case in Spotlight?",
        "answer": "The main elements are the table for exploring data points by their metadata, the inspector for displaying unstructured data, the filter bar for filtering the data, The similarity map as an overview plot on the dataset structure, and potentially multiple additional visualizations such as histograms or scatterplots, as well as metric display elements such as a confusion matrix."
     
    },
    {
        "question": "What is the Inspector and how does it play together with other components?",
        "answer": "It is Spotlight's UI component that is able to display unstructured data such as images, audio files, text, and more. It can be used to easily explore unstructured, multimodal datasets.",
    },
    {
        "question": "What is sliceguard and how does it play together with Spotlight?",
        "answer": "It is a library that can automatically detect data clusters or slices where a machine learning model does not perform well. It uses Spoglight to display its detection results. It is also built by Renumics."
    },
    {
        "question": "I want to look at my audio data. Which possibilities does Spotlight offer?",
        "answer": "You can display audio data in the inspector. Spotlight offers the visualizations audio player and spetrogram. It can also display windows in which a certain event in the audio data occurs. To visualize audio dataset structure you could use audio embeddings that are displayed on the Similarity Map."
    },
    {
        "question": "How can I customize the data visualization in Renumics Spotlight?",
        "answer": "You can configure the UI flexibly via the UI. This involves configuring a layout with different widgets and plots, as well as adding lenses to the inspector widget. You can also save and load configured layouts either via json files or via the python API."
    },
    {
        "question": "Which possibilities for data loading and export does Renumics Spotlight offer?",
        "answer": "Spotlight can load pandas dataframes and huggingface datasets via spotlight.show. It can also load hdf5 files (legacy) and csv files. The preferred way is directly loading pandas dataframes via spotlight.show."
    },
]

In [4]:
import pandas as pd

# Initialize a list to hold your data
data = []

# Iterate over each question
for i, item in enumerate(EXAMPLE_QUESTIONS_WITH_GT):
    print(f"Processing question {i+1}/{len(EXAMPLE_QUESTIONS_WITH_GT)}")
    question = item["question"]

    ground_truth = item["answer"] # Those could be multiple
    
    # Query your RAG system
    response = query_engine.query(question)
    
    # Assuming the response object has the answer
    answer = response.response  # Modify as per your response object structure
    
    # Extract context from the source_nodes attribute
    # Concatenate the text of each source node to form the full context
    contexts = [node.text for node in response.source_nodes]
    
    # Append the details to your data list
    data.append({"question": question, "ground_truth": ground_truth, "answer": answer, "contexts": contexts})

# Convert the list to a pandas DataFrame
df = pd.DataFrame(data)

Processing question 1/11
Processing question 2/11
Processing question 3/11
Processing question 4/11
Processing question 5/11
Processing question 6/11
Processing question 7/11
Processing question 8/11
Processing question 9/11
Processing question 10/11
Processing question 11/11


In [5]:
df

,question,ground_truth,answer,contexts
0,What is the similarity map?,The similarity map is a dimensionality reducti...,The similarity map is a data visualization co...,[---\ntags: []\nid: similarity-map\nsidebar_po...
1,How can I install Renumics Spotlight?,Just run pip install renumics-spotlight. After...,"To install Renumics Spotlight, you can use pi...",[---\nslug: /docs/development\ntitle: Contribu...
2,I want to find duplicates in my data. How can ...,"You can either just do that exploratively, e.g...","To find duplicates in your data, you can use ...",[---\ntags: []\nid: duplicates-annoy\nsidebar_...
3,Which possibilities regarding cluster analysis...,"Spotlight offers the similarity map, which can...",Spotlight offers various possibilities for cl...,[---\nsidebar_position: 40\nslug: /docs/config...
4,I want to identify label issues for my audio u...,"Similarity Map, Inspector and Filtering can he...",To identify label issues for your audio use c...,[---\ntags: []\nid: audio-classification\nslug...
5,What are the main UI elements that help me wit...,The main elements are the table for exploring ...,The main UI elements that can help you with y...,[---\nsidebar_position: 30\nslug: /docs/config...
6,What is the Inspector and how does it play tog...,It is Spotlight's UI component that is able to...,"The Inspector is a widget in Spotlight, a dat...",[---\ntags: []\nid: inspector\nsidebar_positio...
7,What is sliceguard and how does it play togeth...,It is a library that can automatically detect ...,Sliceguard is a library used to identify data...,[---\ntags: []\nid: sliceguard\nsidebar_positi...
8,I want to look at my audio data. Which possibi...,You can display audio data in the inspector. S...,"Based on the provided context information, Sp...","[`audio(column, window_column=None, name=None,..."
9,How can I customize the data visualization in ...,You can configure the UI flexibly via the UI. ...,Renumics Spotlight provides various UI compon...,[---\ntags: []\nid: inspector\nsidebar_positio...


In [6]:
df.to_json("rag_results.json")

